# Launcher

## Setup

In [6]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    run_parallel,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
    storage_path,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)

2022-07-20 12:22:22,404-[INFO]- dropbox:387 -> Refreshing access token.
2022-07-20 12:22:22,712-[INFO]- dropbox:471 -> Request to files/list_folder
2022-07-20 12:22:25,108-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:26,008-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:27,073-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:28,165-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:29,462-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:30,397-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:31,580-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:32,799-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:33,853-[INFO]- dropbox:471 -> Request to files/list_folder/continue
2022-07-20 12:22:35,048-[INFO]- dropbox:471 -> Request to files/list_folder/continue
20

KeyboardInterrupt: 

## Finding the plate

### Chosing from full plate list

In [3]:
all_folders_drop.columns

Index(['folder', 'size', 'change_date', 'tot_path_drop', 'Plate', 'CrossDate',
       'unique_id'],
      dtype='object')

In [4]:
plates = all_folders_drop["unique_id"].unique()
plates

array(['86_20220622', '19_20220525', '1076_20220511', '1070_20220502',
       '1040_20220502', '2_20220411', '1053_20220413', '1025_20220518',
       '1047_20220516', '1035_20220518', '1042_20220523', '1071_20220517',
       '1051_20220518', '22_20220522', '1048_20220506', '32_20220428',
       '13_20220422', '1039_20220516', '4_20220419', '30_20220509',
       '6_20220506', '1016_20220623', '23_20220509', '1015_20220504',
       '27_20220508', '1044_20220509', '1045_20220504', '1033_20220502',
       'faulty_param_files', '28_20220603', '21_2022', '12_20220502',
       '87_20220611', '1023_20220506', '20_20220419', '3_20220426',
       '24_20220426', '1031_20220419', '16_20220419', '1061_20220523',
       '832_20220419', '871_20220419', '1028_20220523', '1019_20220419',
       '877_20220419', '6_20220509', '1032_20220428', '1024_20220516',
       '21_20220502', '31_20220428', '33_20220502', '1023_20220502',
       '824_20220414', '1_20220406', '6666646_20220308', '941_20220404',
     

### Chosing from the crossdate

In [12]:
on_dropbox = all_folders_drop.loc[
    all_folders_drop["CrossDate"].between("20220622", "20220707")
]
plates = on_dropbox["unique_id"].unique()

In [13]:
plates

array(['1016_20220623'], dtype=object)

In [14]:
on_dropbox = on_dropbox.sort_values("folder")

In [15]:
on_dropbox

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
2100,20220623_1558_Plate05,NaN,2022-06-30 13:42:52,data/prince/1016_20220623/20220623_1558_plate05,1016,20220623,1016_20220623
2097,20220623_1658_Plate05,NaN,2022-06-30 13:43:02,data/prince/1016_20220623/20220623_1658_plate05,1016,20220623,1016_20220623
2091,20220623_1813_Plate05,NaN,2022-06-30 13:48:52,data/prince/1016_20220623/20220623_1813_plate05,1016,20220623,1016_20220623
2087,20220623_1913_Plate05,NaN,2022-06-30 13:51:47,data/prince/1016_20220623/20220623_1913_plate05,1016,20220623,1016_20220623
2085,20220623_2013_Plate05,NaN,2022-06-30 13:52:09,data/prince/1016_20220623/20220623_2013_plate05,1016,20220623,1016_20220623
2080,20220623_2113_Plate05,NaN,2022-06-30 13:54:54,data/prince/1016_20220623/20220623_2113_plate05,1016,20220623,1016_20220623
2075,20220623_2213_Plate05,NaN,2022-06-30 13:58:03,data/prince/1016_20220623/20220623_2213_plate05,1016,20220623,1016_20220623
2072,20220623_2313_Plate05,NaN,2022-06-30 14:01:07,data/prince/1016_20220623/20220623_2313_plate05,1016,20220623,1016_20220623
2067,20220624_0013_Plate05,NaN,2022-06-30 14:03:48,data/prince/1016_20220623/20220624_0013_plate05,1016,20220623,1016_20220623
2061,20220624_0113_Plate05,NaN,2022-06-30 14:09:29,data/prince/1016_20220623/20220624_0113_plate05,1016,20220623,1016_20220623


## Download timesteps

In [5]:
# directory_targ = os.path.join(directory_scratch,'stitch_temp')+'/'
plates = ["86_20220622"]

In [6]:
directory_targ = directory_project
directory_project

'/projects/0/einf914/data/'

In [7]:
name_job = f"one_shot_felix{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 30],
    plates,
    "12:00:00",
    name_job=name_job,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

Submitted batch job 1213730


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


## Run operations

In [8]:
run_launcher("skeletonizer.py", [directory_targ], plates, "20:00", dependency=True)

In [6]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "stitcher.py",
    [directory_targ, name_job, 1],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

Submitted batch job 1193321


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [4]:
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, -1],
    plates,
    "20:00",
    dependency=True,
    name_job=name_job,
)

Submitted batch job 1180887


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [3]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/841 [00:00<?, ?it/s]

In [6]:
all_folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220512_...,1045,2,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,"12.05.2022, 17:43:",20220512_1743_Plate02,NaN,1045_20220504,2022-05-12 17:43:00
1,/scratch-shared/amftrack/stitch_temp/20220606_...,4,26,'Carrot','Agg','001P100N100C','Y','-',20220419,25,...,False,True,True,True,True,"06.06.2022, 00:15:",20220606_0015_Plate26,NaN,4_20220419,2022-06-06 00:15:00
2,/scratch-shared/amftrack/stitch_temp/20220514_...,1015,1,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,"14.05.2022, 15:46:",20220514_1546_Plate01,NaN,1015_20220504,2022-05-14 15:46:00
3,/scratch-shared/amftrack/stitch_temp/20220517_...,1023,15,'Carrot','C2','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,"17.05.2022, 15:29:",20220517_1529_Plate15,NaN,1023_20220502,2022-05-17 15:29:00
4,/scratch-shared/amftrack/stitch_temp/20220518_...,33,7,'Carrot','Agg','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,"18.05.2022, 13:17:",20220518_1317_Plate07,NaN,33_20220502,2022-05-18 13:17:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,/scratch-shared/amftrack/stitch_temp/20220521_...,1033,28,'Carrot','C2','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,"21.05.2022, 00:15:",20220521_0015_Plate28,NaN,1033_20220502,2022-05-21 00:15:00
1026,/scratch-shared/amftrack/stitch_temp/20220511_...,1023,15,'Carrot','C2','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,"11.05.2022, 02:44:",20220511_0244_Plate15,NaN,1023_20220502,2022-05-11 02:44:00
1027,/scratch-shared/amftrack/stitch_temp/20220510_...,33,7,'Carrot','Agg','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,"10.05.2022, 09:59:",20220510_0959_Plate07,NaN,33_20220502,2022-05-10 09:59:00
1028,/scratch-shared/amftrack/stitch_temp/20220517_...,1040,22,'Carrot','A5','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,"17.05.2022, 13:48:",20220517_1348_Plate22,NaN,1040_20220502,2022-05-17 13:48:00


In [4]:
# plates= ['12_20220502']
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = all_folders.loc[all_folders["/Analysis/nx_graph_pruned_labeled.p"]==True]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220509_...,1053,39,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"09.05.2022, 05:50:",20220509_0550_Plate39,NaN,1053_20220413,2022-05-09 05:50:00
1,/scratch-shared/amftrack/stitch_temp/20220504_...,1053,39,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"04.05.2022, 20:20:",20220504_2020_Plate39,NaN,1053_20220413,2022-05-04 20:20:00
5,/scratch-shared/amftrack/stitch_temp/20220418_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"18.04.2022, 19:45:",20220418_1945_Plate07,NaN,1053_20220413,2022-04-18 19:45:00
9,/scratch-shared/amftrack/stitch_temp/20220426_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"26.04.2022, 20:36:",20220426_2036_Plate07,NaN,1053_20220413,2022-04-26 20:36:00
13,/scratch-shared/amftrack/stitch_temp/20220421_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"21.04.2022, 12:53:",20220421_1253_Plate07,NaN,1053_20220413,2022-04-21 12:53:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,/scratch-shared/amftrack/stitch_temp/20220512_...,1053,39,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"12.05.2022, 04:51:",20220512_0451_Plate39,NaN,1053_20220413,2022-05-12 04:51:00
806,/scratch-shared/amftrack/stitch_temp/20220428_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,True,True,True,True,True,"28.04.2022, 04:36:",20220428_0436_Plate07,NaN,1053_20220413,2022-04-28 04:36:00
807,/scratch-shared/amftrack/stitch_temp/20220501_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"01.05.2022, 22:58:",20220501_2258_Plate07,NaN,1053_20220413,2022-05-01 22:58:00
808,/scratch-shared/amftrack/stitch_temp/20220420_...,1053,7,'Carrot','C2','001P100N200C','Y','-',20220413,25,...,False,True,True,True,True,"20.04.2022, 22:54:",20220420_2254_Plate07,NaN,1053_20220413,2022-04-20 22:54:00


In [7]:
all_folders["unique_id"].unique()

array(['1053_20220413', '1040_20220502', '4_20220419', '1023_20220502',
       '94_20201123', '938_20220325', '961_20220328'], dtype=object)

In [13]:
i = 0
unique_ids = list(set(folders["unique_id"].values))
unique_ids.sort()
select = folders.loc[folders["unique_id"] == unique_ids[i]]

In [8]:
all_folders
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 14:35:",20220513_1435_Plate17,6_20220509,2022-05-13 14:35:00
1,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 19:35:",20220517_1935_Plate17,6_20220509,2022-05-17 19:35:00
2,/scratch-shared/amftrack/stitch_temp/20220511_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"11.05.2022, 19:50:",20220511_1950_Plate17,6_20220509,2022-05-11 19:50:00
3,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 06:56:",20220517_0656_Plate17,6_20220509,2022-05-17 06:56:00
4,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 10:35:",20220513_1035_Plate17,6_20220509,2022-05-13 10:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 04:32:",20220513_0432_Plate17,6_20220509,2022-05-13 04:32:00
67,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 17:35:",20220517_1735_Plate17,6_20220509,2022-05-17 17:35:00
68,/scratch-shared/amftrack/stitch_temp/20220520_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"20.05.2022, 15:37:",20220520_1537_Plate17,6_20220509,2022-05-20 15:37:00
69,/scratch-shared/amftrack/stitch_temp/20220520_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"20.05.2022, 13:37:",20220520_1337_Plate17,6_20220509,2022-05-20 13:37:00


In [10]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/skeleton_pruned_compressed.mat"] == True]
folders

analysed:   0%|          | 0/71 [00:00<?, ?it/s]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 14:35:",20220513_1435_Plate17,6_20220509,2022-05-13 14:35:00
3,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 06:56:",20220517_0656_Plate17,6_20220509,2022-05-17 06:56:00
4,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 10:35:",20220513_1035_Plate17,6_20220509,2022-05-13 10:35:00
7,/scratch-shared/amftrack/stitch_temp/20220514_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"14.05.2022, 06:33:",20220514_0633_Plate17,6_20220509,2022-05-14 06:33:00
9,/scratch-shared/amftrack/stitch_temp/20220512_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"12.05.2022, 13:50:",20220512_1350_Plate17,6_20220509,2022-05-12 13:50:00
11,/scratch-shared/amftrack/stitch_temp/20220512_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"12.05.2022, 20:32:",20220512_2032_Plate17,6_20220509,2022-05-12 20:32:00
13,/scratch-shared/amftrack/stitch_temp/20220512_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"12.05.2022, 22:32:",20220512_2232_Plate17,6_20220509,2022-05-12 22:32:00
14,/scratch-shared/amftrack/stitch_temp/20220516_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"16.05.2022, 02:37:",20220516_0237_Plate17,6_20220509,2022-05-16 02:37:00
16,/scratch-shared/amftrack/stitch_temp/20220518_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"18.05.2022, 13:48:",20220518_1348_Plate17,6_20220509,2022-05-18 13:48:00
17,/scratch-shared/amftrack/stitch_temp/20220510_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"10.05.2022, 18:51:",20220510_1851_Plate17,6_20220509,2022-05-10 18:51:00


In [7]:
all_folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220606_...,4,26,'Carrot','Agg','001P100N100C','Y','-',20220419,25,...,True,False,True,True,True,True,"06.06.2022, 00:15:",20220606_0015_Plate26,4_20220419,2022-06-06 00:15:00
1,/scratch-shared/amftrack/stitch_temp/20220612_...,1045,25,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,False,"12.06.2022, 00:08:",20220612_0008_Plate25,1045_20220504,2022-06-12 00:08:00
2,/scratch-shared/amftrack/stitch_temp/20220609_...,1033,28,'Carrot','C2','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,False,"09.06.2022, 06:11:",20220609_0611_Plate28,1033_20220502,2022-06-09 06:11:00
3,/scratch-shared/amftrack/stitch_temp/20220613_...,1045,25,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,False,"13.06.2022, 00:08:",20220613_0008_Plate25,1045_20220504,2022-06-13 00:08:00
4,/scratch-shared/amftrack/stitch_temp/20220618_...,1015,31,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,False,"18.06.2022, 23:21:",20220618_2321_Plate31,1015_20220504,2022-06-18 23:21:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,/scratch-shared/amftrack/stitch_temp/20220603_...,12,21,'Carrot','Agg','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,False,"03.06.2022, 08:16:",20220603_0816_Plate21,12_20220502,2022-06-03 08:16:00
207,/scratch-shared/amftrack/stitch_temp/20220605_...,1040,22,'Carrot','A5','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,False,"05.06.2022, 00:06:",20220605_0006_Plate22,1040_20220502,2022-06-05 00:06:00
208,/scratch-shared/amftrack/stitch_temp/20220619_...,1045,25,'Carrot','C2','001P100N200C','Y','-',20220504,25,...,False,False,False,False,False,False,"19.06.2022, 11:06:",20220619_1106_Plate25,1045_20220504,2022-06-19 11:06:00
209,/scratch-shared/amftrack/stitch_temp/20220612_...,1070,23,'Carrot','A5','001P100N200C','Y','-',20220502,25,...,False,False,False,False,False,False,"12.06.2022, 00:02:",20220612_0002_Plate23,1070_20220502,2022-06-12 00:02:00


In [8]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
3,/scratch-shared/amftrack/stitch_temp/20220407_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"07.04.2022, 21:55:",20220407_2155_Plate33,889_20220325,2022-04-07 21:55:00
18,/scratch-shared/amftrack/stitch_temp/20220327_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"27.03.2022, 15:35:",20220327_1535_Plate14,889_20220325,2022-03-27 15:35:00
30,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 21:40:",20220330_2140_Plate14,889_20220325,2022-03-30 21:40:00
39,/scratch-shared/amftrack/stitch_temp/20220419_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"19.04.2022, 01:20:",20220419_0120_Plate33,889_20220325,2022-04-19 01:20:00
62,/scratch-shared/amftrack/stitch_temp/20220328_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"28.03.2022, 05:36:",20220328_0536_Plate14,889_20220325,2022-03-28 05:36:00
64,/scratch-shared/amftrack/stitch_temp/20220415_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"15.04.2022, 05:55:",20220415_0555_Plate33,889_20220325,2022-04-15 05:55:00
86,/scratch-shared/amftrack/stitch_temp/20220401_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"01.04.2022, 18:45:",20220401_1845_Plate14,889_20220325,2022-04-01 18:45:00
98,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 23:40:",20220330_2340_Plate14,889_20220325,2022-03-30 23:40:00
101,/scratch-shared/amftrack/stitch_temp/20220329_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"29.03.2022, 21:10:",20220329_2110_Plate14,889_20220325,2022-03-29 21:10:00
112,/scratch-shared/amftrack/stitch_temp/20220418_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"18.04.2022, 01:20:",20220418_0120_Plate33,889_20220325,2022-04-18 01:20:00


In [12]:
for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

Submitted batch job 1138954


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [20]:
plates

['6_20220509']

In [19]:
select

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


In [13]:
i = 0
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

Matching plate 20220510_1851_Plate17 at dates ['20220510_1851_Plate17', '20220511_1750_Plate17']


In [17]:
from pymatreader import read_mat

from amftrack.pipeline.functions.image_processing.extract_graph import (
    sparse_to_doc,
    get_degree3_nodes,
)
import open3d as o3d
from cycpd import rigid_registration
import pandas as pd
import os

dilateds = []
skels = []
skel_docs = []
for directory_name in dates:
    path_snap = os.path.join(directory_targ, directory_name)
    skel_info = read_mat(path_snap + "/Analysis/skeleton_pruned.mat")
    skel = skel_info["skeleton"]
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)

In [18]:
skel_docs

[{(9706, 41696): 1,
  (9706, 41697): 1,
  (9706, 41698): 1,
  (9707, 41694): 1,
  (9707, 41695): 1,
  (9708, 41692): 1,
  (9708, 41693): 1,
  (9708, 41694): 1,
  (9709, 41690): 1,
  (9709, 41691): 1,
  (9709, 41692): 1,
  (9710, 41690): 1,
  (9711, 41688): 1,
  (9711, 41689): 1,
  (9712, 41685): 1,
  (9712, 41686): 1,
  (9712, 41687): 1,
  (9713, 41682): 1,
  (9713, 41683): 1,
  (9713, 41684): 1,
  (9714, 41679): 1,
  (9714, 41680): 1,
  (9714, 41681): 1,
  (9715, 41677): 1,
  (9715, 41678): 1,
  (9716, 41675): 1,
  (9716, 41676): 1,
  (9716, 41677): 1,
  (9717, 41674): 1,
  (9717, 41675): 1,
  (9718, 41673): 1,
  (9718, 41674): 1,
  (9719, 41672): 1,
  (9720, 41670): 1,
  (9720, 41671): 1,
  (9721, 41666): 1,
  (9721, 41667): 1,
  (9721, 41668): 1,
  (9721, 41669): 1,
  (9722, 41663): 1,
  (9722, 41664): 1,
  (9722, 41665): 1,
  (9723, 41660): 1,
  (9723, 41661): 1,
  (9723, 41662): 1,
  (9724, 41658): 1,
  (9724, 41659): 1,
  (9725, 41655): 1,
  (9725, 41656): 1,
  (9725, 41657): 1,


## Local launcher

In [10]:
model_name = "test"
directory = os.path.join(storage_path, "groundtruths") + "/"

In [11]:
update_plate_info(directory)
folder_df = get_current_folders(directory)

analysed:   0%|          | 0/26 [00:00<?, ?it/s]

In [12]:
folder_df

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:05:",20220714_1405_Plate18,120focus_20220701,2022-07-14 14:05:00
1,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:09:",20220714_1409_Plate18,120focus_20220701,2022-07-14 14:09:00
2,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:13:",20220714_1413_Plate18,120focus_20220701,2022-07-14 14:13:00
3,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:17:",20220714_1417_Plate18,120focus_20220701,2022-07-14 14:17:00
4,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:22:",20220714_1422_Plate18,120focus_20220701,2022-07-14 14:22:00
5,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:26:",20220714_1426_Plate18,120focus_20220701,2022-07-14 14:26:00
6,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:30:",20220714_1430_Plate18,120focus_20220701,2022-07-14 14:30:00
7,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:34:",20220714_1434_Plate18,120focus_20220701,2022-07-14 14:34:00
8,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:38:",20220714_1438_Plate18,120focus_20220701,2022-07-14 14:38:00
9,/media/kahane/AMFtopology02/storage/groundtrut...,120focus,18,'Carrot','C2','001P100N100C','Y','-',20220701,25,...,False,False,False,False,False,False,"14.07.2022, 14:44:",20220714_1444_Plate18,120focus_20220701,2022-07-14 14:44:00


In [ ]:
selected_df = folder_df.loc[folder_df["folder"] == plate_name]
i = 0
exp = Experiment(directory)
exp.load(selected_df, suffix="")
exp.load_tile_information(0)

In [15]:
from amftrack.pipeline.launching.run import run_stitch

In [17]:
run_stitch(os.path.join(storage_path, "groundtruths"), folder_df)

stitched:   0%|          | 0/26 [00:00<?, ?it/s]

/home/ipausers/kahane/opt/fiji-linux64/Fiji.app/ImageJ-linux64 --mem=8000m --headless --ij2 --console -macro /data/felix/TEMP/stitching_loops/stitching_loop1658312904905173003.ijm


Java HotSpot(TM) 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
Java HotSpot(TM) 64-Bit Server VM warning: Using incremental CMS is deprecated and will likely be removed in a future release


20220714_1405_Plate18/
(Wed Jul 20 12:38:07 CEST 2022): Stitching the following files:
Image: '/media/kahane/AMFtopology02/storage/groundtruths/20220714_1405_Plate18//Img2/Img_r01_c01.tif' Imp: 'null' Offset: (0.0, 0.0)
Image: '/media/kahane/AMFtopology02/storage/groundtruths/20220714_1405_Plate18//Img2/Img_r01_c02.tif' Imp: 'null' Offset: (3276.0, 0.0)
Image: '/media/kahane/AMFtopology02/storage/groundtruths/20220714_1405_Plate18//Img2/Img_r01_c03.tif' Imp: 'null' Offset: (6552.0, 0.0)
Image: '/media/kahane/AMFtopology02/storage/groundtruths/20220714_1405_Plate18//Img2/Img_r01_c04.tif' Imp: 'null' Offset: (9828.0, 0.0)
Image: '/media/kahane/AMFtopology02/storage/groundtruths/20220714_1405_Plate18//Img2/Img_r01_c05.tif' Imp: 'null' Offset: (13104.0, 0.0)
Image: '/media/kahane/AMFtopology02/storage/groundtruths/20220714_1405_Plate18//Img2/Img_r01_c06.tif' Imp: 'null' Offset: (16380.0, 0.0)
Image: '/media/kahane/AMFtopology02/storage/groundtruths/20220714_1405_Plate18//Img2/Img_r01_c07.t

0 overlaps 1: 0.0 translation: x: 2156 y: 23
0 overlaps 15: 0.0 translation: x: -5280 y: 2509
0 overlaps 16: 0.0 translation: x: 2156 y: 2505
1 overlaps 2: 0.0 translation: x: 2156 y: 23
1 overlaps 15: 0.0 translation: x: -4396 y: 2505
1 overlaps 16: 0.0 translation: x: -5280 y: 2509
1 overlaps 17: 0.0 translation: x: 2156 y: 2505
2 overlaps 3: 0.0 translation: x: 2156 y: 23
2 overlaps 16: 0.0 translation: x: -4396 y: 2505
2 overlaps 17: 0.0 translation: x: -5280 y: 2509
2 overlaps 18: 0.0 translation: x: 2527 y: 2149
3 overlaps 4: 0.0 translation: x: 2387 y: 580
3 overlaps 17: 0.0 translation: x: -4396 y: 2505
3 overlaps 18: 0.0 translation: x: -4565 y: 2782
3 overlaps 19: 0.0 translation: x: 2479 y: 2623
4 overlaps 5: 0.3194563764135358 translation: x: 3167 y: -35
4 overlaps 18: 0.5754237741660466 translation: x: -3230 y: 2465
4 overlaps 19: 0.33366606186166264 translation: x: -50 y: 2417
4 overlaps 20: 0.10773062214649298 translation: x: 3249 y: 2345
5 overlaps 6: 0.2587581693053469

...

38 overlaps 39: 0.7642903761515467 translation: x: 3215 y: -44
38 overlaps 52: 0.7327486454311977 translation: x: -3184 y: 2442
38 overlaps 53: 0.7748103899473069 translation: x: 27 y: 2396
38 overlaps 54: 0.5403347064535468 translation: x: 3260 y: 2359
39 overlaps 40: 0.7054689647765147 translation: x: 3230 y: -45
39 overlaps 53: 0.6130427511725302 translation: x: -3198 y: 2447
39 overlaps 54: 0.6629253351605668 translation: x: 35 y: 2396
39 overlaps 55: 0.629329679501283 translation: x: 3267 y: 2351
40 overlaps 41: 0.6869182272199852 translation: x: 3225 y: -40
40 overlaps 54: 0.6942618688129096 translation: x: -3188 y: 2436
40 overlaps 55: 0.7697007100871364 translation: x: 35 y: 2396
40 overlaps 56: 0.7353703542526867 translation: x: 3261 y: 2352
41 overlaps 42: 0.8109167731244433 translation: x: 3258 y: -44
41 overlaps 55: 0.7209370688200785 translation: x: -3191 y: 2443
41 overlaps 56: 0.8311685184583467 translation: x: 33 y: 2397
41 overlaps 57: 0.8406760779079545 translation: x

73 overlaps 74: 0.0 translation: x: 2585 y: -515
73 overlaps 87: 0.3676613336376856 translation: x: -3224 y: 2449
73 overlaps 88: 0.8588325168845876 translation: x: 32 y: 2406
73 overlaps 89: 0.7281611668552272 translation: x: 3291 y: 2363
74 overlaps 88: 0.0 translation: x: -2354 y: 2148
74 overlaps 89: 0.0 translation: x: 4501 y: 2475
75 overlaps 76: 0.8280170160626671 translation: x: 3139 y: -43
75 overlaps 90: 0.9000854608684346 translation: x: 35 y: 2403
75 overlaps 91: 0.8157388073001306 translation: x: 3173 y: 2355
76 overlaps 77: 0.6113123387337194 translation: x: 3248 y: -43
76 overlaps 90: 0.7701013591458236 translation: x: -3102 y: 2441
76 overlaps 91: 0.7552847023946068 translation: x: 36 y: 2403
76 overlaps 92: 0.5780590369821942 translation: x: 3284 y: 2351
77 overlaps 78: 0.6650448871611191 translation: x: 3240 y: -44
77 overlaps 91: 0.5579852892531306 translation: x: -3213 y: 2440
77 overlaps 92: 0.7245832049696188 translation: x: 35 y: 2403
77 overlaps 93: 0.5524780554

107 overlaps 122: 0.8326993534557076 translation: x: 28 y: 2426
107 overlaps 123: 0.5993729580464858 translation: x: 3271 y: 2377
108 overlaps 109: 0.4339813866788517 translation: x: 3242 y: -46
108 overlaps 122: 0.552415526762121 translation: x: -3215 y: 2475
108 overlaps 123: 0.8202383886669472 translation: x: 28 y: 2433
108 overlaps 124: 0.4775913642770607 translation: x: 3272 y: 2385
109 overlaps 110: 0.6383982039130195 translation: x: 3238 y: -46
109 overlaps 123: 0.2934112978862086 translation: x: -3214 y: 2480
109 overlaps 124: 0.7646119386393018 translation: x: 28 y: 2427
109 overlaps 125: 0.7277787774690188 translation: x: 3264 y: 2380
110 overlaps 111: 0.6280457760405732 translation: x: 3226 y: -47
110 overlaps 124: 0.634773242831089 translation: x: -3210 y: 2470
110 overlaps 125: 0.7789009482669396 translation: x: 28 y: 2423
110 overlaps 126: 0.6337931469396078 translation: x: 3263 y: 2380
111 overlaps 112: 0.7046894345566985 translation: x: 3220 y: -44
111 overlaps 125: 0.6

Successfully optimized configuration of 128 tiles after 1974 iterations:
  average displacement: 3.966px
  minimal displacement: 2.186px
  maximal displacement: 6.778px
 image information list size =128
(Wed Jul 20 21:13:33 CEST 2022): Final image positions in the fused image:
Tile 4 (/media/kahane/AMFtopology02/storage/groundtruths/20220714_1602_Plate18//Img2/Img_r01_c05.tif): 0.0, 0.0
Tile 5 (/media/kahane/AMFtopology02/storage/groundtruths/20220714_1602_Plate18//Img2/Img_r01_c06.tif): 3149.5654, -41.75232
Tile 6 (/media/kahane/AMFtopology02/storage/groundtruths/20220714_1602_Plate18//Img2/Img_r01_c07.tif): 6391.58, -85.99951
Tile 7 (/media/kahane/AMFtopology02/storage/groundtruths/20220714_1602_Plate18//Img2/Img_r01_c08.tif): 9618.985, -128.56909
Tile 8 (/media/kahane/AMFtopology02/storage/groundtruths/20220714_1602_Plate18//Img2/Img_r01_c09.tif): 12832.292, -171.90027
Tile 9 (/media/kahane/AMFtopology02/storage/groundtruths/20220714_1602_Plate18//Img2/Img_r01_c10.tif): 16060.699, -

(Wed Jul 20 21:20:06 CEST 2022): Finished Stitching.
Saving /media/kahane/AMFtopology02/storage/groundtruths/20220714_1602_Plate18//Stitched Image_1.tif ... 
Saved
Image grid sequence stitching is done!
1
